In [ ]:
#This code learn ranking based on fairness def of CTR, and evaluate based on fairness def of CTR with trust bias (Compas data)

import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
import numpy as np
import pickle as pkl
import pickle
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from YahooDataReader import YahooDataReader
import math
import statistics
import itertools
from random import seed, shuffle
import os,sys
import urllib.request  as urllib2
from collections import defaultdict
from sklearn import preprocessing
from IPython.display import display
from collections import Counter

# Preprocessing

In [ ]:
SEED = 1234
seed(SEED)
np.random.seed(SEED)

def add_intercept(x):

    """ Add intercept to the data before linear classification """
    m,n = x.shape
    intercept = np.ones(m).reshape(m, 1) # the constant b
    return np.concatenate((intercept, x), axis = 1)

In [ ]:
def load_compas_data():

    FEATURES_CLASSIFICATION = ["age_cat", "race", "sex", "priors_count", "c_charge_degree"] #features to be used for classification
    CONT_VARIABLES = ["priors_count"] # continuous features, will need to be handled separately from categorical features, categorical features will be encoded using one-hot
    CLASS_FEATURE = "two_year_recid" # the decision variable
    SENSITIVE_ATTRS = ["race"]


    COMPAS_INPUT_FILE = "../data/compas.csv"  # Comas data address

    # load the data and get some stats
    df = pd.read_csv(COMPAS_INPUT_FILE)
    df
    df = df.dropna(subset=["days_b_screening_arrest"]) # dropping missing vals

    # convert to np array
    data = df.to_dict('list')
    for k in data.keys():
        data[k] = np.array(data[k])


    """ Filtering the data """

    # These filters are the same as propublica (refer to https://github.com/propublica/compas-analysis)
    # If the charge date of a defendants Compas scored crime was not within 30 days from when the person was arrested, we assume that because of data quality reasons, that we do not have the right offense. 
    idx = np.logical_and(data["days_b_screening_arrest"]<=30, data["days_b_screening_arrest"]>=-30)


    # We coded the recidivist flag -- is_recid -- to be -1 if we could not find a compas case at all.
    idx = np.logical_and(idx, data["is_recid"] != -1)

    # In a similar vein, ordinary traffic offenses -- those with a c_charge_degree of 'O' -- will not result in Jail time are removed (only two of them).
    idx = np.logical_and(idx, data["c_charge_degree"] != "O") # F: felony, M: misconduct

    # We filtered the underlying data from Broward county to include only those rows representing people who had either recidivated in two years, or had at least two years outside of a correctional facility.
    idx = np.logical_and(idx, data["score_text"] != "NA")

    # we will only consider blacks and whites for this analysis
    idx = np.logical_and(idx, np.logical_or(data["race"] == "African-American", data["race"] == "Caucasian"))

    # select the examples that satisfy this criteria
    for k in data.keys():
        data[k] = data[k][idx]

    #display(data)

    """ Feature normalization and one hot encoding """

    # convert class label 0 to -1
    y = data[CLASS_FEATURE]
    #y[y==0] = -1



    print ("\nNumber of people recidivating within two years")
    print (pd.Series(y).value_counts())
    print ("\n")


    X = np.array([]).reshape(len(y), 0) # empty array with num rows same as num examples, will hstack the features to it
    x_control = defaultdict(list)

    feature_names = []
    for attr in FEATURES_CLASSIFICATION:
        print ("attr", attr)
        vals = data[attr]
        if attr in CONT_VARIABLES:
            vals = [float(v) for v in vals]
            vals = preprocessing.scale(vals) # 0 mean and 1 variance  
            vals = np.reshape(vals, (len(y), -1)) # convert from 1-d arr to a 2-d arr with one col

        else: # for binary categorical variables, the label binarizer uses just one var instead of two
            lb = preprocessing.LabelBinarizer()
            lb.fit(vals)
            vals = lb.transform(vals)
    
        # add to sensitive features dict
        if attr in SENSITIVE_ATTRS:
            print ("True")
            x_control[attr] = vals


        # add to learnable features
        X = np.hstack((X, vals))

        if attr in CONT_VARIABLES: # continuous feature, just append the name
            feature_names.append(attr)
        else: # categorical features
            if vals.shape[1] == 1: # binary features that passed through lib binarizer
                feature_names.append(attr + "_" + str(lb.classes_[1]))
            else:
                for k in lb.classes_: # non-binary categorical features, need to add the names for each cat
                    feature_names.append(attr + "_" + str(k))


    # convert the sensitive feature to 1-d array
    x_control = dict(x_control)
    for k in x_control.keys():
        assert(x_control[k].shape[1] == 1) # make sure that the sensitive feature is binary after one hot encoding
        x_control[k] = np.array(x_control[k]).flatten()
    print ("x_control", x_control)
    # sys.exit(1)

    #"""permute the date randomly"""
    #perm = list(range(0,X.shape[0])
    #shuffle(perm)
    #X = X[perm]
    #y = y[perm]
    #for k in x_control.keys():
    #x_control[k] = x_control[k][perm]


    X = add_intercept(X)

    feature_names = ["intercept"] + feature_names
    assert(len(feature_names) == X.shape[1])
    print ("Features we will be using for classification are:", feature_names, "\n")

    # wrtie data in csv file

    #A = pd.DataFrame(list(x_control.items()),columns=['index','race'])
    #A.to_csv('compas_A.csv', sep='\t') 
    y = pd.DataFrame(data=y,columns=['two_year_recid'])
    y.to_csv('compas_Y.csv', sep='\t') 
    X1 = pd.DataFrame(data=X[0:,0:],columns=feature_names) #X[0,1:] #index=X[0:,0]
    #print ("X1B", X1.keys())
    A = X1.race_Caucasian
    #print ("A", A)
    X1.drop(['race_Caucasian'], axis=1) #in place-- does not work.So X1 is unchanged
    print ("X1", X1.keys())
    #X = pd.DataFrame(list(X.items()),columns=feature_names)
    X1.to_csv('compas_X.csv', sep='\t')
    A.to_csv('compas_A.csv', sep='\t')
    data = pd.concat([A,X1],axis = 1).values #race_caucasian column is added to ll data. So this feature appears twice, once at index=0 and also at index=5. Here, we consider cocuasians to be bottom rank in criminal search
    #print ("data", data)
    return data, y


In [ ]:
def make_sample_record(orig_data, orig_label):

    #data, to_protect, label = load_compas_data() #read_arrhythmia(shuffle=True, dropmissing=True)

    data = orig_data
    orig_label = np.array(orig_label)
    label = orig_label

    data = pd.DataFrame(data)
    label = pd.DataFrame(label)
    display(data)


    Y = np.array(label)
    Y = np.squeeze(Y)

    #data = pd.concat([to_protect,data],axis = 1).values

    X = np.array(data)

    num_feats = X.shape[1]
    numX = X.shape[0]
    #print("num_feat: ", num_feats)
    #print("Y: ", Y)

    datasize = 500
    cs_size = 25
    split_on_doc = 0.8
    testsize = 100
    ratio_relevant = 0.4
    ratios_col = Y * ratio_relevant + (1-Y)*(1-ratio_relevant)

    # generate a candidate set of size 10 everytime
    data_X = []
    data_Y = []
    test_X = []
    test_Y = []
    group_identities_train = []
    group_identities_test = []
    print("Sampling between 0 and {} for train".format(numX*split_on_doc))
    p = ratios_col[0:int(numX*split_on_doc)]
    p = p / sum(p)
    for i in range(datasize):
        cs_indices = np.random.choice(np.arange(0, int(numX*split_on_doc), dtype=int), size=cs_size, p=p)
        cs_X = X[cs_indices]
        cs_Y = Y[cs_indices]
        data_X.append(cs_X)
        data_Y.append(cs_Y)
        group_identities_train.append(cs_X[:,0]) #index=0 shows caucasian= whites
    print("Sampling between {} and {} for test".format(int(numX*split_on_doc), numX))
    p = ratios_col[int(numX*split_on_doc):]
    p = p/sum(p)
    docID_test= []
    #for i in range(testsize): 
    while len(docID_test) < testsize: #used while instead of for to make sure given "continue", we still have len(testsize) qid#
        cs_indices = np.random.choice(np.arange(numX*split_on_doc, numX, dtype=int), size=cs_size, p=p)
        print ("cs_indices",cs_indices)
        #*START: added to skip the sets that has zero female or male or all female rel score 0 or all male relscore 0*#
        group_id= X[cs_indices,0]
        print ("group_id", group_id)
        if (len(set(group_id))==1 or 
            sum([Y[item] for i, item in enumerate(cs_indices) if group_id[i]==0]) == 0 or
            sum([Y[item] for i, item in enumerate(cs_indices) if group_id[i]==1]) == 0):
            #print ("continue")
            continue
        #*END: added to skip the sets that has zero female or male*#
        docID_test.append (cs_indices.tolist())
        print ("docID_test", docID_test)
        test_X.append(X[cs_indices])
        test_Y.append(Y[cs_indices])
        group_identities_test.append(X[cs_indices,0])


    return data_X, data_Y, test_X, test_Y, docID_test

In [ ]:
data, label = load_compas_data()
data_X, data_Y, test_X, test_Y, docID_test= make_sample_record(data, label)
pkl.dump((data_X, data_Y), open("compas_train_rank_25.pkl", "wb"))
pkl.dump((test_X, test_Y), open("compas_test_rank_25.pkl", "wb"))
pickle.dump(docID_test, open( "compas_docID_test_25.txt", "wb" ) )


In [ ]:
data_df = pd.DataFrame(data)
MaleID= data_df[data_df[0]==1].index
FemaleID= data_df[data_df[0]==0].index
MaleID_str=[]
FemaleID_str=[]
for i in MaleID:
    MaleID_str.append(str(i))
for i in FemaleID:
    FemaleID_str.append(str(i))

In [ ]:
# read data
docID_test = pickle.load( open( "compas_docID_test_25.txt", "rb" ) )

with open('compas_train_rank_25.pkl', 'rb') as f:
    data_train = pickle.load(f)

with open('compas_test_rank_25.pkl', 'rb') as f:
     data_test = pickle.load(f)

In [ ]:
# learn rel scores
dr = YahooDataReader(None)
dr.data = pkl.load(open("compas_train_rank_25.pkl", "rb"))
vdr = YahooDataReader(None)
vdr.data = pkl.load(open("compas_test_rank_25.pkl","rb"))                  
model = linear_model.LinearRegression(fit_intercept=False) #, normalize=False
feats, rel = dr.data
feats = np.array([item for sublist in feats for item in sublist])
rel = np.array([item for sublist in rel for item in sublist])
model.fit(feats, rel)
# predictions on validation
feats, rel = vdr.data
se_sum = 0
length = 0
predicted_rels = []
for i, query in enumerate(feats):
    rel_pred = model.predict(query)    
    predicted_rels.append(rel_pred)
    se_sum += np.sum((rel_pred - rel[i])**2)
    length += len(rel[i])

# Fair Ranking

In [ ]:

docID_test_str=[[] for _ in range(len(docID_test))] #type=str
size_female_test = []
size_male_test=[]
sum_rel_female=[]
sum_rel_male= []
sum_rel_totall=[]
sum_rel_female_predicted=[]
sum_rel_male_predicted= []
sum_rel_totall_predicted=[]


for qid in range(len(docID_test)):
    F_size= 0
    M_size= 0
    F_rel=0
    M_rel=0
    F_rel_predicted=0
    M_rel_predicted=0
    for doc in range(len(docID_test[0])):
        docID_test_str[qid].append(str(docID_test[qid][doc]))
        if str(docID_test[qid][doc]) in FemaleID_str:
            F_size=F_size+1
            F_rel= F_rel+ data_test[1][qid][int (float(docID_test_str[qid][doc]))] # used only for evaluation (ML1M)
            F_rel_predicted= F_rel_predicted+ predicted_rels[qid][int (float(docID_test_str[qid][doc]))] #used in FORGE and LinkedIn algo (ML1M)
            
        else:
            M_size= M_size+1
            M_rel= M_rel+ data_test[1][qid][int (float(docID_test_str[qid][doc]))]
            M_rel_predicted= M_rel_predicted+ predicted_rels[qid][int (float(docID_test_str[qid][doc]))] 
    
    size_female_test.append(F_size) 
    size_male_test.append(M_size)
    totall_size= len (docID_test_str[qid])
    
    sum_rel_female.append(F_rel) 
    sum_rel_male.append(M_rel)
    sum_rel_totall.append(sum_rel_female[qid]+ sum_rel_male[qid])
    
    sum_rel_female_predicted.append(F_rel_predicted) 
    sum_rel_male_predicted.append(M_rel_predicted)
    sum_rel_totall_predicted.append(sum_rel_female_predicted[qid]+ sum_rel_male_predicted[qid])

for i in range(len(np.divide(sum_rel_female, size_female_test))):
    if np.divide(sum_rel_female_predicted, size_female_test)[i] > np.divide(sum_rel_male_predicted, size_male_test)[i]:
        print ("f")

In [ ]:
def KL_Divergence(p1,p2,q1,q2): #can use jenson divergance that is bounded between 0 and 1#
    if p1!=0 and p2!=0:
        J=(    (p1*np.log((2*p1)/(p1+q1)))  +  (p2*np.log((2*p2)/(p2+q2)))    +  (q1*np.log((2*q1)/(p1+q1)))  +  (q2*np.log((2*q2)/(p2+q2)))    ) / 2
    elif p1==0:
        J=(    (p2*np.log((2*p2)/(p2+q2)))    +  (q1*np.log((2*q1)/(p1+q1)))  +  (q2*np.log((2*q2)/(p2+q2)))    ) / 2
    elif p2==0:
        J=(   (p1*np.log((2*p1)/(p1+q1)))  +  (q1*np.log((2*q1)/(p1+q1)))  +  (q2*np.log((2*q2)/(p2+q2)))    ) / 2
    else:
        J=(   (q1*np.log((2*q1)/(p1+q1)))  +  (q2*np.log((2*q2)/(p2+q2)))   )/ 2 
    return J

In [ ]:
def fairness(size_docID, 
             size_female, 
             size_male, 
             current_male_CTR, 
             sofar_male_CTR, 
             current_female_CTR, 
             sofar_female_CTR,
             ):


    CTR_male = sofar_male_CTR + current_male_CTR
    CTR_female =  sofar_female_CTR + current_female_CTR
    CTR_total = CTR_male + CTR_female
    current_fairness= 1- KL_Divergence(CTR_male / CTR_total,
                                       CTR_female / CTR_total,
                                       size_male/size_docID, 
                                       size_female/size_docID
                                       )
    return current_fairness

In [ ]:
def separate_and_sort(
    docID,
    rel_scores
    ):
    """
    Arg list:
    movieID: list of movie IDs. Here, list of all docID for a certain qid
    rel_scores: dict() that maps movie_ID -> rel_score
    """
    female_sorted_by_rel = []
    male_sorted_by_rel = []
    
    for i in docID:    
        if i in FemaleID_str:
            female_sorted_by_rel.append(i)
        else:
            male_sorted_by_rel.append(i)

    female_sorted_by_rel.sort(key = lambda x : rel_scores[x], reverse=True) 
    male_sorted_by_rel.sort(key = lambda x : rel_scores[x], reverse=True)

    return male_sorted_by_rel, female_sorted_by_rel

In [ ]:

def interpolation_optimized(
    qid,
    docID,
    rel_scores,
    Z,
):
    """
    Arg list:
    docID: list of docIDs for each qid in test data
    rel_scores: dict() that maps doc_ID -> rel_score
    Z: Z value for interpolation
    """
    all_sorted_by_rel= sorted(docID, key = lambda x : rel_scores[x], reverse=True)
    S=[]
    sofar_female_CTR=0
    sofar_male_CTR=0
    sofar_DCG=0
    IDCG=0
    
    availablity = all_sorted_by_rel[:] #make a copy of all_sorted to avoid del make problem for IDCG
    while len (S)< 30:
        IDCG= IDCG+ (float(2**float(rel_scores[all_sorted_by_rel[len(S)]])-1) /  math.log2(1+len(S)+1))
        epsilon_1= 0.65
        epsilon_plus= 1- ((len(S)+2)/100)
        epsilon_minus= epsilon_1* (1/min( len(S)+1, 10) )
        max_intpol_score = 0
        max_item_data = None
        for item in availablity:
            DCG = sofar_DCG+ (float(2**float(rel_scores[item])-1) /  math.log2(1+len(S)+1))
            nDCG =  float (DCG)/ IDCG
            if item in FemaleID_str:
                current_male_CTR = 0
                current_female_CTR = float(  (rel_scores[item]* epsilon_plus) + ( (1-rel_scores[item]) * epsilon_minus)  ) * 1/math.log2(1+len(S)+1)
            else:
                current_male_CTR =   float(  (rel_scores[item]* epsilon_plus) + ( (1-rel_scores[item]) * epsilon_minus)  ) * 1/math.log2(1+len(S)+1)
                current_female_CTR = 0
            fair_metric = fairness(sum_rel_totall_predicted[qid], sum_rel_female_predicted[qid], sum_rel_male_predicted[qid], current_male_CTR, sofar_male_CTR, current_female_CTR, sofar_female_CTR)
            intpol_score = (1-Z) * nDCG + Z * fair_metric
            if intpol_score > max_intpol_score:
                max_intpol_score = intpol_score
                max_item_data = (item, current_female_CTR, current_male_CTR, DCG)



        S.append(max_item_data[0])
        availablity.remove(max_item_data[0])
        sofar_female_CTR += max_item_data[1]
        sofar_male_CTR += max_item_data[2]
        sofar_DCG = max_item_data[3]
        
    return S

In [ ]:

def get_nDCG(docID, rel_scores, item_list):
    """
    gives the nDCG of ranking
    input:
    docID: id of documents
    rel_scores: relevant scores assuming predict = ground truth or not
    item_list: dictionary key= docID, value= relscores
    """
    sorted_docID= sorted(docID, key=lambda x:rel_scores [int(x)] , reverse=True) #for ML1M
    #print ("sorted_docID", sorted_docID)
    Denom= float(0)
    Nom= float(0)
    for i in range (len(item_list)):
        temp1= D_real[str(sorted_docID[i])] #D1 if want to assume predicted rel is ground truth, D_real otherwise#
        temp2= 2**(float (temp1))
        Denom= Denom+(   (temp2-1)  / (math.log2(i+2))    )
        temp3= D_real[str(item_list[i])]    #D1 if want to assume predicted rel is ground truth, D_real otherwise#
        temp4= 2**(float (temp3))
        Nom=Nom + (  (temp4-1)  / (math.log2(i+2))    )
    nDCG= (float(Nom)/float(Denom))
    return (nDCG, sorted_docID  )

In [ ]:
def get_divergence (qid, docID, rel_scores):
    sofar_female_CTR=0
    sofar_male_CTR=0
    D_movie_list=[]
    D_male_list=[]
    D_female_list=[]
    for i in range(len(docID)):
        epsilon_1= 0.65
        epsilon_plus= 1- ((i+2)/100)
        epsilon_minus= epsilon_1* ( 1/min (i+1, 10) )
        if docID[i] in FemaleID_str:
            sofar_female_CTR += float(  (rel_scores[docID[i]]* epsilon_plus) + ( (1-rel_scores[docID[i]]) * epsilon_minus)  ) * 1/math.log2(i+2)
        else:
            sofar_male_CTR +=   float(  (rel_scores[docID[i]]* epsilon_plus) + ( (1-rel_scores[docID[i]]) * epsilon_minus)  ) * 1/math.log2(i+2)
        
        CTR_total= sofar_female_CTR+ sofar_male_CTR  
        D_movie_list.append(KL_Divergence(sofar_male_CTR/ CTR_total, sofar_female_CTR/ CTR_total , sum_rel_male[qid]/sum_rel_totall[qid], sum_rel_female[qid]/sum_rel_totall[qid]))
        D_male_list.append(float((sofar_male_CTR/ CTR_total)/(sum_rel_male[qid]/sum_rel_totall[qid])))
        D_female_list.append(float((sofar_female_CTR/ CTR_total)/(sum_rel_female[qid]/sum_rel_totall[qid])))
    return (D_movie_list, D_male_list, D_female_list)

In [ ]:
def get_divergence_trust_DP (qid, docID, rel_scores):
    sofar_female_CTR=0
    sofar_male_CTR=0
    D_movie_list=[]
    for i in range(len(docID)):
        epsilon_1= 0.65
        epsilon_plus= 1- ((i+2)/100)
        epsilon_minus= epsilon_1* ( 1/min (i+1, 10) )
        if docID[i] in FemaleID_str:
            sofar_female_CTR += float(  (rel_scores[docID[i]]* epsilon_plus) + ( (1-rel_scores[docID[i]]) * epsilon_minus)  ) * 1/math.log2(i+2)
        else:
            sofar_male_CTR +=   float(  (rel_scores[docID[i]]* epsilon_plus) + ( (1-rel_scores[docID[i]]) * epsilon_minus)  ) * 1/math.log2(i+2)
        
        CTR_total= sofar_female_CTR+ sofar_male_CTR  
        D_movie_list.append(KL_Divergence(sofar_male_CTR/ CTR_total, sofar_female_CTR/ CTR_total , size_male_test[qid]/totall_size, size_female_test[qid]/totall_size))
    return (D_movie_list)

In [ ]:
def get_divergence_pos_DT (qid, docID, rel_scores):
    sofar_female_exp=0
    sofar_male_exp=0
    D_movie_list=[]
    for i in range(len(docID)):
        if docID[i] in FemaleID_str:
            sofar_female_exp +=  1/math.log2(i+2)
        else:
            sofar_male_exp +=  1/math.log2(i+2)
        
        exp_total= sofar_female_exp+ sofar_male_exp  
        D_movie_list.append(KL_Divergence(sofar_male_exp/ exp_total, sofar_female_exp/ exp_total , sum_rel_male[qid]/sum_rel_totall[qid], sum_rel_female[qid]/sum_rel_totall[qid]))
    return (D_movie_list)

In [ ]:
def get_divergence_pos_DP (qid, docID, rel_scores):
    sofar_female_exp=0
    sofar_male_exp=0
    D_movie_list=[]
    for i in range(len(docID)):
        if docID[i] in FemaleID_str:
            sofar_female_exp +=  1/math.log2(i+2)
        else:
            sofar_male_exp +=  1/math.log2(i+2)
        
        exp_total= sofar_female_exp+ sofar_male_exp  
        D_movie_list.append(KL_Divergence(sofar_male_exp/ exp_total, sofar_female_exp/ exp_total , size_male_test[qid]/totall_size, size_female_test[qid]/totall_size))
    return (D_movie_list)

In [ ]:
#This function is fairness definition of joachims (difference) based on disparate treatment

def get_joachims_diff (qid, docID):  #gives a vector that has conv at each rank indx. can feed both item_list and sorted_item_list#
    
        
    sofar_female_expo=0
    sofar_male_expo=0
    sofar_indx_expo=0
    Diff_movie_list=[]
    for i in range(len(docID)):
        if docID[i] in FemaleID_str:
            sofar_female_expo= sofar_female_expo+(1/math.log2(i+2)) 
        else:
            sofar_male_expo= sofar_male_expo + (1/math.log2(i+2))   
        sofar_indx_expo= sofar_indx_expo+ (1/math.log2(i+2))
        if sum_rel_male[qid]/size_male_test[qid] > sum_rel_female[qid]/size_female_test[qid]:
            sign= 1
        else:
            sign=-1
        Diff_movie_list.append(max(0, sign*diff(sofar_male_expo/ size_male_test[qid], sofar_female_expo/ size_female_test[qid] , sum_rel_male[qid]/size_male_test[qid], sum_rel_female[qid]/size_female_test[qid])))

    return (Diff_movie_list)

In [ ]:
def get_CPFair_diff (qid, docID):  #gives a vector that has conv at each rank indx. can feed both item_list and sorted_item_list#
    
        
    sofar_female_count=0
    sofar_male_count=0
    Diff_movie_list=[]
    for i in range(len(docID)):
        if docID[i] in FemaleID_str:
            sofar_female_count= sofar_female_count+1 
        else:
            sofar_male_count= sofar_male_count + 1   
        Diff_movie_list.append(abs(sofar_male_count- sofar_female_count))

    return (Diff_movie_list)

In [ ]:
def get_FairMetric(DocID, DivOrDiff_item_list):    #do not need it if we have new trust and CTR
    Denom=float(0)
    Nom=float(0)
    for i in range (len(DocID)):
        Denom= Denom+ (1/ (math.log2(i+2)))
        Nom= Nom+ (1/ (math.log2(i+2)))*DivOrDiff_item_list[i]   
    n_item_list= float(Nom)/ float(Denom)
    return (n_item_list)

In [ ]:

import math

interplotion_value=[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.91, 0.92, 0.94, 0.96, 0.98, 0.99, 1]
for z in interplotion_value:
    D2={} # a dictionary: key= user id, value= S/item_list

    nDCG_all_q=[]
    D_item_list_all_q=[]
    male_Div_all_q=[]
    female_Div_all_q=[]
    
    D_item_list_trust_DP_all_q=[]
    D_item_list_pos_DT_all_q=[]
    D_item_list_pos_DP_all_q=[]
    joachims_diff_item_list_all_q=[]
    CPFair_diff_item_list_all_q=[]
    

    



    
    
    for qid in range(len(docID_test)):
        nDCG=[]
        nDiv=[]
        D1={} # a dictionary: key= movieID, value= predicted rel_scores
        D_real= {} # a dictionary: key= movieID, value= true rel_scores
        for i in range (len(docID_test[0])):
            #D1[docID_test_str[qid][i]]= predicted_rels[qid][i] # for German Compas Comcast and ML100K
            #D_real[docID_test_str[qid][i]]= data_test[1][qid][i] # for German Compas Comcast and ML100K
            
            D1[docID_test_str[qid][i]]= predicted_rels[qid][int (float(docID_test_str[qid][i]))] # for ML1M
            D_real[docID_test_str[qid][i]]= data_test[1][qid][int (float(docID_test_str[qid][i]))] # for ML1M

        item_list  = interpolation_optimized (qid,docID_test_str[qid], D1, z)
        for i in range(len(item_list)):
            nDCG.append(get_nDCG(docID_test[qid], data_test[1][qid], item_list[0:i+1])[0]) #predicted_rels[qid] if assume precietd rel is ground truth, data_test[1][qid] Otherwise
            D_item_list= get_divergence(qid, item_list[0:i+1], D_real)[0]  #D1 if we assume predicted=true rel, and D_real o.w
            male_Div= get_divergence(qid, item_list[0:i+1], D_real)[1]
            female_Div= get_divergence(qid, item_list[0:i+1], D_real)[2]
            
            D_item_list_trust_DP= get_divergence_trust_DP (qid, item_list[0:i+1], D_real)
            D_item_list_pos_DT= get_divergence_pos_DT (qid, item_list[0:i+1], D_real)
            D_item_list_pos_DP= get_divergence_pos_DP (qid, item_list[0:i+1], D_real)
            joachims_diff_item_list= get_joachims_diff (qid, item_list[0:i+1])
            CPFair_diff_item_list= get_CPFair_diff (qid, item_list[0:i+1])
            
            


        nDCG_all_q.append (nDCG)
        D_item_list_all_q.append(D_item_list)
        male_Div_all_q.append(male_Div)
        female_Div_all_q.append(female_Div)
        
        D_item_list_trust_DP_all_q.append(D_item_list_trust_DP)
        D_item_list_pos_DT_all_q.append(D_item_list_pos_DT)
        D_item_list_pos_DP_all_q.append(D_item_list_pos_DP)
        joachims_diff_item_list_all_q.append(joachims_diff_item_list)
        CPFair_diff_item_list_all_q.append(CPFair_diff_item_list)

        


    avg_nDCG_at_indx_all_q= np.mean(nDCG_all_q, axis=0)
    avg_D_item_list_at_indx_all_q= np.mean(D_item_list_all_q, axis=0)
    avg_male_Div_at_indx_all_q= np.mean(male_Div_all_q, axis=0)
    avg_female_Div_at_indx_all_q= np.mean(female_Div_all_q, axis=0)
    
    avg_D_item_list_trust_DP_at_indx_all_q= np.mean(D_item_list_trust_DP_all_q, axis=0)
    avg_D_item_list_pos_DT_at_indx_all_q= np.mean(D_item_list_pos_DT_all_q, axis=0)
    avg_D_item_list_pos_DP_at_indx_all_q= np.mean(D_item_list_pos_DP_all_q, axis=0)
    avg_joachims_diff_item_list_at_indx_all_q= np.mean(joachims_diff_item_list_all_q, axis=0)
    avg_CPFair_diff_item_list_at_indx_all_q= np.mean(CPFair_diff_item_list_all_q, axis=0)
    

    print ("K=5")
    print (avg_nDCG_at_indx_all_q[4], end='\t')
    print (avg_D_item_list_at_indx_all_q[4], end='\t')
    print (avg_male_Div_at_indx_all_q[4], end='\t')
    print (avg_female_Div_at_indx_all_q[4], end='\t')
    print (avg_D_item_list_trust_DP_at_indx_all_q[4], end='\t')
    print (avg_D_item_list_pos_DT_at_indx_all_q[4], end='\t')
    print (avg_D_item_list_pos_DP_at_indx_all_q[4], end='\t')
    print (avg_joachims_diff_item_list_at_indx_all_q[4], end='\t')
    print (avg_CPFair_diff_item_list_at_indx_all_q[4], end='\t')
    print ("********************")
    print ("********************")
    print ("K=10")
    print ( avg_nDCG_at_indx_all_q[9], end='\t')
    print ( avg_D_item_list_at_indx_all_q[9], end='\t')
    print ( avg_male_Div_at_indx_all_q[9], end='\t')
    print ( avg_female_Div_at_indx_all_q[9], end='\t')
    print (avg_D_item_list_trust_DP_at_indx_all_q[9], end='\t')
    print (avg_D_item_list_pos_DT_at_indx_all_q[9], end='\t')
    print (avg_D_item_list_pos_DP_at_indx_all_q[9], end='\t')
    print ( avg_joachims_diff_item_list_at_indx_all_q[9], end='\t')
    print ( avg_CPFair_diff_item_list_at_indx_all_q[9], end='\t')
    print ("********************")
    print ("********************")
    print ("K=30")
    print ( avg_nDCG_at_indx_all_q[29], end='\t')
    print ( avg_D_item_list_at_indx_all_q[29], end='\t')
    print ( avg_male_Div_at_indx_all_q[29], end='\t')
    print ( avg_female_Div_at_indx_all_q[29], end='\t')
    print ( avg_D_item_list_trust_DP_at_indx_all_q[29], end='\t')
    print ( avg_D_item_list_pos_DT_at_indx_all_q[29], end='\t')
    print ( avg_D_item_list_pos_DP_at_indx_all_q[29], end='\t')
    print ( avg_joachims_diff_item_list_at_indx_all_q[29], end='\t')
    print ( avg_CPFair_diff_item_list_at_indx_all_q[29], end='\t')
    print ("********************")
    print ("********************")